Reference:
- https://www.youtube.com/watch?v=KfnhNlD8WZI
- https://www.youtube.com/watch?v=7n1SpeudvAE

In [ ]:
import pandas as pd
import numpy as np

# Scikit-Learn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.externals import joblib

# Tensorflow
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers impor1t Dense

In [ ]:
OUT_PATH_FILE = 'resources/output/'

In [ ]:
def load_data():
    dataset = datasets.load_iris()
    df = pd.DataFrame(dataset['data'], columns=dataset['feature_names'])
    df['target'] = dataset['target']
    df['target name'] = df['target'].map({i: x for i,x in enumerate(dataset['target_names'])})
    
    return df

In [ ]:
# Data loading
df = load_data()

df.shape

# Model Training

In [ ]:
def feature_target_split(df):
    X = df[[x for x in df.columns if 'target' not in x]].copy()
    y = df['target'].copy()
    
    return X, y

In [ ]:
# Feature-target separation
X, y = feature_target_split(df)

X.shape, y.shape

In [ ]:
y.value_counts()

In [ ]:
def encode_target(y):
    return pd.get_dummies(y)

def decode_target(y):
    return np.argmax(y, axis=1)

In [ ]:
y = encode_target(y)

y.shape

In [ ]:
def train_test(X, y, test_size=.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0, stratify=y)

    X_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    
    return X_train, X_test, y_train, y_test

In [ ]:
# Split training & testing dataset
X_train, X_test, y_train, y_test = train_test(X, y)

X_train.shape, X_test.shape

In [ ]:
# Pre-processing pipeline
steps    = [
    ('std_scaler', StandardScaler()),
    ('mm_scaler', MinMaxScaler())
]
pipeline = Pipeline(steps, verbose=True)
X_train  = pipeline.fit_transform(X_train)

X_train.shape

In [ ]:
# Neural Network
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.fit(X_train, y_train, epochs=100)

In [ ]:
# Apply transformation to test dataset
X_test = pipeline.transform(X_test)

X_test.shape

In [ ]:
# Prediction on test dataset
y_pred = model.predict(X_test)
y_pred = decode_target(y_pred)
y_test = decode_target(y_test.values)

y_pred.shape, y_test.shape

In [ ]:
def classif_eval(y_true, y_pred):
    cofmat_df = pd.DataFrame(confusion_matrix(y_true, y_pred))
    cofmat_df.index.name   = 'True'
    cofmat_df.columns.name = 'Pred'

    print(cofmat_df)
    print()
    print(classification_report(y_true, y_pred, digits=5))

In [ ]:
# Model evaluation
classif_eval(y_test, y_pred)

In [ ]:
# Save scikit-learn models
for key, value in pipeline.named_steps.items():
    joblib.dump(value, f'{OUT_PATH_FILE}{key}.joblib')

# Save tensorflow model
model.save(f'{OUT_PATH_FILE}classif_model.H5')

# Model Restoration

In [ ]:
# Feature-target separation
X, y = feature_target_split(df)

X.shape, y.shape

In [ ]:
# Split training & testing dataset
X_train, X_test, y_train, y_test = train_test(X, y)

X_train.shape, X_test.shape

In [ ]:
# Restore models
std_scaler = joblib.load(f'{OUT_PATH_FILE}std_scaler.joblib')
mm_scaler  = joblib.load(f'{OUT_PATH_FILE}mm_scaler.joblib')
model      = load_model(f'{OUT_PATH_FILE}classif_model.H5')

In [ ]:
# Pre-processing pipeline
steps    = [
    ('std_scaler', std_scaler),
    ('mm_scaler', mm_scaler)
]
pipeline = Pipeline(steps, verbose=True)
X_test  = pipeline.transform(X_test)

In [ ]:
# Prediction
y_pred = model.predict(X_test)
y_pred = decode_target(y_pred)

y_pred.shape

In [ ]:
# Model evaluation
classif_eval(y_test, y_pred)